# Lesson 2 Exercise 3: Creating Fact and Dimension Tables with Star Schema


In [1]:
import psycopg2

In [2]:
conn=psycopg2.connect('host=127.0.0.1 dbname=data_engineering user=postgres password=macbook')
cur=conn.cursor()


In [3]:
conn.set_session(autocommit=True)

### Imagine you work at an online Music Store. There will be many tables in our database, but let's just focus on 4 tables around customer purchases. 

<img src="images/starSchema.png" width="750" height="750">

### From this representation you can start to see the makings of a "STAR". You will have one fact table (the center of the star) and 3  dimension tables that are coming from it.

### Create Fact table and insert data

In [4]:
cur.execute("create table if not exists customer_transactions(customer_id int,store_id int,spent numeric)")
query="insert into customer_transactions(customer_id,store_id,spent)values(%s,%s,%s)"

In [6]:
cur.execute(query,(1,1,20.50))
cur.execute(query,(2,1,35.21))

## Create dimension tables

In [7]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS customers (customer_id integer, name varchar, rewards boolean)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('INSERT INTO customers (customer_id, name, rewards) \
                VALUES (%s, %s, %s)', \
               (1, 'Amanda', True))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('INSERT INTO customers (customer_id, name, rewards) \
                VALUES (%s, %s, %s)', \
               (2, 'Toby', False))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS purchased_items (customer_id integer, item_number integer, item_name varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('INSERT INTO purchased_items (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)', \
               (1, 1, 'Rubber Soul'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
try: 
    cur.execute('INSERT INTO purchased_items (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)', \
               (2, 3, 'Let it be'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS stores (store_id integer, state varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
try: 
    cur.execute('INSERT INTO stores (store_id, state) \
                VALUES (%s, %s)', \
               (1, 'CA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute('INSERT INTO stores (store_id, state) \
                VALUES (%s, %s)', \
               (2, 'WA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### Now run the following queries on this data easily because of utilizing the Fact/ Dimension and Star Schema
 
#### Query 1: Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.

#### Query 2: How much did Customer 2 spend?

In [9]:
cur.execute("select * from customer_transactions join customers on customers.customer_id=customer_transactions.customer_id join purchased_items on purchased_items.customer_id=customers.customer_id join stores on stores.store_id=customer_transactions.store_id where customer_transactions.spent > 30 ")

In [10]:
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(2, 1, Decimal('35.21'), 2, 'Toby', False, 2, 3, 'Let it be', 1, 'CA')


### Your output from the above cell should look like this:
('Toby', 1, 'CA', 'Let It Be', False)

#### Query 2: How much did Customer 2 spend?

In [13]:
cur.execute("select customer_id,sum(spent) from customer_transactions where customer_id=2 group by customer_id")

In [14]:
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(2, Decimal('35.21'))


### Your output from the above cell should include Customer 2 and the amount: 
(2, 35.21)

### Drop the tables

In [15]:
try: 
    cur.execute("DROP TABLE customer_transactions")
    cur.execute("DROP TABLE purchased_items")
    cur.execute("DROP TABLE customers")
    cur.execute("DROP TABLE stores")
    
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

### Close the connections 

In [16]:
conn.close()
cur.close()